Данное домашнее задание посвящено схеме базе данных, связанной с футбольным командами. Схема расположена в hw_diagram.jpg
## Описание таблиц 

### Positions 
Позиции игроков
id - уникальный идентификатор позиции 
Name - Название таблицы 

### Coaches 
Таблица тренеров 
id - уникальный идентификатор тренера
Name - имя тренера 

## Teams 
Таблица команд 
id - уникальный идентификатор  команды 
Name - имя команды 
Location - расположение команды 
Coach_id - идентификатор тренера 

### Players 
Таблица игроков 
Id - уникальный идентификатор игрока 
Name - имя игрока 
Age - возраст игрока 
Position_id - идентификатор позиции 
Team_id - идентификатор команды 
Start_date - дата начала игры в команде 

### PlayersArchive 
Архив по игрокам (информация о том, в какой команде игрок играл ранее) 
player_id - идентификатор игрока 
team_id - идентификатор команды 
start_date - дата начала игры 
end_date - дата окончания игры

# Задание 1
Реализовать перенакатываемые sql-скрипты по схеме hw_diagram, приложенного ниже.

In [13]:
%reload_ext sql
%sql sqlite:///football.db

'Connected: None@football.db'

In [14]:
import sqlite3

In [15]:
%%sql
pragma foreign_key=ON;
drop table if exists players_arсhive;
drop table if exists players;
drop table if exists teams;
drop table if exists position;
drop table if exists coaches;
create table coaches(id integer primary key, name text);

create table position(id integer primary key, name text);

create table teams(id integer primary key, name text, location text, coach_id integer, foreign key(coach_id) references coaches(id));

create table players(id integer primary key, name text, age integer, team_id integer, position_id integer, start_date data,
                        foreign key(team_id) references teams(id), foreign key(position_id) references position(id));

create table players_arсhive(player_id integer, team_id integer, start_date data, end_date data,
    foreign key(player_id) references players(id), foreign key(team_id) references teams(id), primary key(player_id, team_id));

Done.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.


[]

In [16]:
%sql select * from players_arсhive

Done.


player_id,team_id,start_date,end_date


# Задание 2
Написать sql-скрипт, осуществляюший добавление всех возможных позиций игроков. (В данной задаче можно ограничится четырьмя - вратарь, защитник, полузащитник, нападающий)

In [17]:
%%sql
Insert into position values (10, 'вратарь');
Insert into position values (20, 'защитник');
Insert into position values (30, 'полузащитник');
Insert into position values (40, 'нападающий');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [18]:
%sql select * from position

Done.


id,name
10,вратарь
20,защитник
30,полузащитник
40,нападающий


# Задание 3
Написать функцию, осуществляющую добавление тренера. На вход функции подается имя тренера. При невозможности вставки функция должна обработать данное исключение и вывести пользователю информацию о проблеме

После написания функции необходимо добавить несколько тренеров для таблицы.

In [19]:
db=sqlite3.connect('football.db')
def check_coach(name1):
    cur1=db.cursor()
    cur1.execute(''' select id from coaches where name=?''', (name1,))
    try:
        coa_id=cur1.fetchone()[0]
        print ('Такой тренер уже есть')
        return -1
    except TypeError:
        print ('Добавлено')
        return 1


def insert_coach(name_coach):
    coach_id=check_coach(name_coach)
    if coach_id==-1:
        return -1
    else:
        cur2=db.cursor()
        cur2.execute(''' select coalesce(max(id)+1,1) as new_id from coaches ''')
        new_id=cur2.fetchone()[0]
        cur2.execute(''' insert into coaches(id, name) values (?,?) ''', (new_id, name_coach,))
db.commit()

In [20]:
insert_coach('Фред Грим')
insert_coach('Фред Грим')
insert_coach('С.С.Черчесов')
insert_coach('Тите')

Добавлено
Такой тренер уже есть
Добавлено
Добавлено


In [21]:
%sql select * from coaches

Done.


id,name


# Задание 4

Написать функцию, осуществляющую добавление команды. На вход функция принимает (имя команды, местоположение, имя тренера).
На выходе должна быть добавлена команда.

После написания функции необходимо добавить несколько команд

In [22]:
def insert_team(team_name, its_loc, its_coache):
    cur4=db.cursor()
    cur4.execute(''' select coalesce(max(id)+1,1) as new_id from teams ''')
    new_id=cur4.fetchone()[0]
    cur4.execute(''' insert into teams(id, name, location, coach_id) values (?,?,?,?) ''', (new_id, team_name, its_loc, its_coache))
db.commit()

In [23]:
insert_team('Сборная России', 'Россия', 'С.С.Черчесов')
insert_team('Сборная Голландии', 'Нидерланды', 'Фред Грим')
insert_team('Сборная Бразилии', 'Бразилия', 'Тите')

In [24]:
%sql select * from teams

Done.


id,name,location,coach_id


# Задание 5

Написать функцию, осуществляющую обновление информации о команде. У функции на входе должен быть один обязательный параметр - название команды и два необязательных параметра - имя тренера и местоположение. При этом хотя бы один из необязательных параметров должен быть заполнен. Функция должна обновлять значение переданных необязательных параметров.

После написания функции вызовите данную функцию несколько раз

In [25]:
def check_team(name1):
    cur1=db.cursor()
    cur1.execute(''' select id from teams where name=?''', (name1,))
    try:
        t_id=cur1.fetchone()[0]
        return t_id
    except TypeError:
        print ('Нет такой команды')
        return -1


def update_teams_location(name_team, its_loc):
    try_team_id=check_team(name_team)
    if try_team_id==-1:
        return -1
    cur=db.cursor()
    cur.execute(''' update teams set location=? where name=?''', (its_loc, name_team,))
db.commit()

In [26]:
update_teams_location('Сборная Голландии', 'Голландия')
update_teams_location('Сборная России', 'РФ')

In [27]:
%sql select * from teams

Done.


id,name,location,coach_id
1,Сборная России,Россия,С.С.Черчесов
2,Сборная Голландии,Нидерланды,Фред Грим
3,Сборная Бразилии,Бразилия,Тите


# Задание 6

Написать функцию, осуществляющую добавление нового футболиста в таблицу игроков. На входе (имя, возраст, название позиции, название команды, дата перехода в команду(необязательный параметр)). Если дата перехода не указана, то нужно использовать текущую дату. Предусмотреть различные ошибки (такие как неверное имя позиции, неверное название команды, возраст < 10 и > 50 и т.п.)

После написания функции необходимо добавить несколько игроков.

In [28]:
def check_player(name1):
    cur1=db.cursor()
    cur1.execute(''' select id from players where name=?''', (name1,))
    try:
        pl_id=cur1.fetchone()[0]
        return pl_id
    except TypeError:
        print ('Добавление')
        return -1

def check_team(name2):
    cur2=db.cursor()
    cur2.execute(''' select id from teams where id=?''', (name2,))
    try:
        t_id=cur2.fetchone()[0]
        return t_id
    except TypeError:
        print ('Нет такой команды')
        return -1
    
def check_position(name3):
    cur3=db.cursor()
    cur3.execute(''' select id from position where id=?''', (name3,))
    try:
        p_id=cur3.fetchone()[0]
        return p_id
    except TypeError:
        print ('Нет такой позиции')
        return -1    
    
def insert_player(player_name, age, teamId, positionId, date):
    cur=db.cursor()
    player_id=check_player(player_name)
    team_id=check_team(teamId)
    position_id=check_position(positionId)
    if (player_id!=-1 and team_id==-1 and position_id==-1 or age>50 or age<10):
        print('Ошибка')
        return -1
    cur.execute(''' select coalesce(max(id)+1,1) as new_id from teams ''')
    new_id=cur.fetchone()[0]
    cur.execute(''' insert into players(id, name, age, team_id, position_id, start_date) values (?,?,?,?,?,?)''', (new_id, player_name, age, team_id, position_id, date))
    db.commit()

In [29]:
insert_player('Пеле', 76, 3, 40, '1957-04-10')
insert_player('Паулиньо', 28, 3, 30, '2011-09-15')
insert_player('Велтман', 25, 2, 20, '2013-10-21')
insert_player('Акинфеев', 31, 1, 10, '2004-04-28')

Добавление
Ошибка
Добавление
Добавление


IntegrityError: UNIQUE constraint failed: players.id

In [30]:
%sql select * from players

Done.


id,name,age,team_id,position_id,start_date
4,Паулиньо,28,3,30,2011-09-15


# Задание 7
Написать функцию для перевода игрока в другую команду. Данная функция должна осуществить архивирование текущей записи в таблице игроков в таблицу players_archive с заполнение end_date - датой начала игры за новую команду - 1 день. А также осуществлять изменение поля в таблице Players. При возникновении ошибки должен осуществляться полный откат транзакции.

In [33]:
def check_team(name1):
    cur1=db.cursor()
    cur1.execute(''' select id from teams where id=?''', (name1,))
    try:
        t_id=cur1.fetchone()[0]
        return t_id
    except TypeError:
        print ('Нет такой команды')
        return -1

def update_players_arсhive_and_team(id_player, id_team, new_date):
    try_team_id=check_team(id_team)
    if try_team_id==-1:
        return -1
    cur=db.cursor()
    cur.execute(''' select end_date from players_arсhive where player_id=?''', (id_player,))
    EndDate=cur.fetchone()[0]
    cur.execute(''' update players_arсhive set team_id=?, start_date=?, end_date=? where id=?''', (id_team, EndDate, new_date, id_player,))
    cur.execute(''' update players set team_id=?, start_date=? where id=?''', (id_team, new_date, id_player,))
db.commit()

In [32]:
update_players_arсhive_and_team(2, 3, '2017-05-08')

TypeError: 'NoneType' object is not subscriptable